[<< back to main index](../README.md)

# Multiple Linear Regression Lab : House Prices

### Overview
Estimate house prices using MLR

### Builds on
None

### Run time
approx. 20 minutes

### Notes



In [ ]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
spark


## Step 1 : House data

In [ ]:
%%time

housePrices = spark.read.csv("/data/house-prices/house-sales-full.csv", \
                             header=True, inferSchema=True)
print("read {:,} records".format(housePrices.count()))


In [ ]:
housePrices.printSchema()
housePrices.limit(5).toPandas().T

Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot" **

In [ ]:
# TODO: select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"
housePrices_small = housePrices.select("???", "???", "???", "???", "???")
housePrices_small.show()

## Step 2: Create Features and Label Column in Dataframe

We will create our "features" column, which is of type vector, and our label column, for which we can just reuse the SquareFootage column.





In [ ]:
from pyspark.ml.feature import VectorAssembler

## TODO : Build features with  "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"
## Hint : inputCols = ['Bedrooms', 'Bathrooms', ??? , ??? ]

assembler = VectorAssembler(inputCols=[???], outputCol="features")
featureVector = assembler.transform(housePrices_small)

## TODO : create a label column to mirror 'SalePrice'
featureVector = featureVector.withColumn("label", featureVector["???"])
featureVector.show()

##  Step 3: Run the Linear Regression Model


In [ ]:
from pyspark.ml.regression import LinearRegression

## Hint : Run model with max iterations as 10
#lr = LinearRegression(maxIter=???)
lr = LinearRegression(maxIter=???, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(featureVector)

print("Coefficents:" + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))



**Insepct the coeffiients.  Are you seeing any ZERO (0) values?  What is the meaning of this?**

In [ ]:
print("numIterations: %d" % lrModel.summary.totalIterations)
print("objectiveHistory: %s" % str(lrModel.summary.objectiveHistory))
# lrModel.summary.residuals.show()
print("RMSE: %f" % lrModel.summary.rootMeanSquaredError)

## TODO : print 'r2' from training summary
## Hint : Use TAB completion
print("r2: %f" % lrModel.summary.???)


#### Observe the summary output
- How many iterations did it do?
- Is it more than our 'max' iterations?
- If so, increase the 'maxIter' to 50 and run it again
- How many iterations is done now?  
- Can you explain the behavior?
- Also did the coefficients change from  maxIter=10  to maxIter=50 ?
- Did r2 change?  Why / Why-not?

## Step 4:  Predict some home prices
Create a new data frame with the following data

In [ ]:
import pandas as pd

# Make a Pandas dataframe
new_data = pd.DataFrame({'Bedrooms' : [5,3,2],
                         'Bathrooms' : [3,2,1.5],
                         'SqFtTotLiving' : [4400, 1800, 1550],
                         'SqFtLot' : [10000, 5000, 4000]
             })

new_data_spark = spark.createDataFrame(new_data)
new_featureVector = assembler.transform(new_data_spark)  #run VectorAssembler again to create features.
new_featureVector.show()




## Step 5: Run on some prediction data

Lets see the predicted output.

In [ ]:
## TODO: transform dataframe with new feature vector to get predictions
## Hint : new_featureVector

predicted_prices = lrModel.transform(???)
predicted_prices.show()

## Step 6 : Some more FUN
Now let's feed 'SalePrice' as an input to our VectorAssembler.  
In Step-2, try this

```python
assembler = VectorAssembler(inputCols=["SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"],
                            outputCol="features")
```

Run again, and observe the following:
- look at the coefficients  (why are most of them zero?)
- look at r2

Can you explain what is going on :-) 